### Carga de datos

En esta sección configuramos las rutas de los repositorios y cargamos los conjuntos de datos (dataset de consultas y corpus de documentos).  Utilizamos la librería datasets de Hugging Face. Si los datos ya están descargados en disco, se cargan desde el disco.

In [25]:
repo = 'spanish-ir/'
d = { 'dataset': 'messirve', 'corpus': 'eswiki_20240401_corpus' }

In [26]:
import os.path
import datasets

dataset, corpus = None, None

if not os.path.isdir('dataset'):
    revision = '1.2'
    country = 'full'
    dataset = datasets.load_dataset(repo + d['dataset'],
        revision=revision, country=country)

if not os.path.isdir('corpus'):
    corpus = datasets.load_dataset(repo + d['corpus'])

Si los objetos no se descargaron en el paso anterior (porque ya existían), los cargamos desde el disco local. 

In [27]:
# in case we already have the dataset, load them from disk
dataset = dataset if dataset is not None else datasets.load_from_disk('dataset')
train, test = dataset['train'].to_pandas(), dataset['test'].to_pandas()

corpus = corpus if corpus is not None else datasets.load_from_disk('corpus')
corpus = corpus['corpus'].to_pandas()

Una vez tenemos los conjuntos de test, train y el corpus, podemos comprobar su estructura, en especial las columnas y su contenido.

In [4]:
train.info()

<class 'pandas.DataFrame'>
RangeIndex: 766296 entries, 0 to 766295
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               766296 non-null  int64  
 1   query            766296 non-null  str    
 2   docid            766296 non-null  str    
 3   docid_text       766296 non-null  str    
 4   docid_title      766296 non-null  str    
 5   query_date       766296 non-null  object 
 6   answer_date      766296 non-null  object 
 7   match_score      766296 non-null  float32
 8   expanded_search  766296 non-null  bool   
 9   answer_type      766296 non-null  str    
 10  id_country       766296 non-null  float64
dtypes: bool(1), float32(1), float64(1), int64(1), object(2), str(5)
memory usage: 463.5+ MB


In [5]:
train.head(3)

,id,query,docid,docid_text,docid_title,query_date,answer_date,match_score,expanded_search,answer_type,id_country
0,7397857,cuántos inning se juegan en el kickingball,1869086#17,El juego de kitball tiene 6 entradas y cada un...,Kickball,2024-04-07,2024-05-06,0.8829,False,feat_snip,976827.0
1,7397858,cómo beneficia la biodiversidad a la salud de...,16208#36,La biodiversidad es importante ya que cada esp...,Biodiversidad,2024-04-06,2024-05-09,1.0000,False,feat_snip,976828.0
2,7397861,quienes somos,3328953#1,"Wikipedia es una enciclopedia libre, políglota...",Wikipedia,2024-05-18,2024-06-24,1.0000,False,feat_snip,6328791.0


In [6]:
test.info()

<class 'pandas.DataFrame'>
RangeIndex: 174078 entries, 0 to 174077
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               174078 non-null  int64  
 1   query            174078 non-null  str    
 2   docid            174078 non-null  str    
 3   docid_text       174078 non-null  str    
 4   docid_title      174078 non-null  str    
 5   query_date       174078 non-null  object 
 6   answer_date      174078 non-null  object 
 7   match_score      174078 non-null  float32
 8   expanded_search  174078 non-null  bool   
 9   answer_type      174078 non-null  str    
 10  id_country       174078 non-null  float64
dtypes: bool(1), float32(1), float64(1), int64(1), object(2), str(5)
memory usage: 105.9+ MB


In [7]:
test.head(3)

,id,query,docid,docid_text,docid_title,query_date,answer_date,match_score,expanded_search,answer_type,id_country
0,7397859,en grecia quién aplico la democracia radical,87525#2,Efialtes es considerado por muchos historiador...,Efialtes de Atenas,2024-04-07,2024-05-06,1.0,False,feat_snip,976830.0
1,7397860,que conoces de la familia arduino,1337914#0,"Arduino es una compañía de desarrollo de ""soft...",Arduino,2024-05-20,2024-06-20,1.0,False,feat_snip,5870869.0
2,7397866,1 arroba cuantas kilogramos tiene,77666#7,En Bolivia y Perú hojas de coca se comercializ...,Arroba (unidad de masa),2024-04-09,2024-05-07,1.0,False,feat_snip,976874.0


In [8]:
corpus.info()

<class 'pandas.DataFrame'>
RangeIndex: 14047759 entries, 0 to 14047758
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   docid   str  
 1   title   str  
 2   text    str  
dtypes: str(3)
memory usage: 5.3 GB


In [9]:
corpus.head(3)

,docid,title,text
0,7#0,Andorra,"Para otros usos de este término, véase Andorra..."
1,7#1,Andorra,"Andorra, oficialmente Principado de Andorra ()..."
2,7#2,Andorra,"Con sus 468 km² de extensión territorial, Ando..."


### Filtrado y limpieza inicial

Para optimizar el uso de memoria, eliminamos columnas del dataset que no son necesarias para la búsqueda (fechas, metadatos, etc.).

In [ ]:
columns = ['id', 'docid_text', 'query_date', 'answer_date', 'expanded_search', 'answer_type', 'id_country']
train.drop(columns=columns, inplace=True)
test.drop(columns=columns,  inplace=True)

A continuación se filtra el corpus para conservar únicamente los documentos que aparecen en los conjuntos de entrenamiento o prueba.

In [29]:
import gc

# select only the documents that are present in the dataset
docids = set(train['docid']) | set(test['docid'])
subcorpus = corpus[corpus['docid'].isin(docids)].copy()

# free memory
del corpus
gc.collect()

930

### Preprocesado

Definimos dos estrategias de preprocesamiento según el método de recuperación que vayamos a usar: 

- Para ``TF-IDF``: Se hará uso de una limpieza *fuerte* que incluye eliminación de stopwords y normalización para reducir el ruido.

- Para ``Embeddings``: Se hará una limpieza *suave*, donde solo se pasará a minúsculas y eliminarán los signos de puntuación, ya que los modelos semánticos necesitan el contexto de las palabras funcionales.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
stops = set(stopwords.words('spanish'))

def preprocess_tfidf(text: str) -> str:
    """ Applies strong preprocessing to a text """
    text = text.lower()
    # remove punctuation signs
    text = re.sub(r'[^\w\s]','', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    # remove stopwords
    words = [w for w in text.split() if w not in stops]
    return ' '.join(words)


def preprocess_embeddings(text: str) -> str:
    """ Applies light preprocessing to a text """
    text = text.lower()
    # remove punctuation signs
    text = re.sub(r'[^\w\s]','', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    return text

Ahora, usamos joblib para paraleizar el procesamiento de los textos y acelerar la tarea a la hora de preprocesar.

In [ ]:
from joblib import Parallel, delayed

def parallel_apply(series, f):
    # applies a function to a pandas series in parallel
    return Parallel(n_jobs=-1)(delayed(f)(x) for x in series)

Una vez definda la función de paralelización, preprocesamos las columnas necesarias del conjunto de test y train.

In [53]:
for column in ['query', 'docid_title']:
    # apply the preprocessing to the dataset columns, for embeddings
    train[column + '_raw'] = parallel_apply(train[column], preprocess_embeddings)
    test[column + '_raw'] = parallel_apply(test[column], preprocess_embeddings)

    # apply the preprocessing to the dataset columns, for tf-idf
    train[column + '_tfidf'] = parallel_apply(train[column], preprocess_tfidf)
    test[column + '_tfidf'] = parallel_apply(test[column], preprocess_tfidf)

De la misma manera, se procesa el corpus.

In [55]:
for column in ['text', 'title']:
    # apply the preprocessing to the corpus columns for embeddings & tf-idf
    subcorpus[column + '_raw'] = parallel_apply(subcorpus[column], preprocess_embeddings)
    subcorpus[column + '_tfidf'] = parallel_apply(subcorpus[column], preprocess_tfidf)

### Recuperación con TF-IDF

En esta sección se utilizará el enfoque clásico de Bag of Words. Para ello, en primer lugar se vectorizará el corpus y las consultas y después, se calculará la similitud coseno entre cada consulta y los documentos. 

### Vectorizado de documentos y consultas

Configuramos el TfidfVectorizer para que trabaje con unigramas y bigramas, y limitamos el vocabulario a 50,000 características para mantener el manejo de memoria viable. 

En un caso en el que se disponga de mayor capacidad de cómputo, usar ngramas de una mayor dimensionalidad (p.ej. trigramas) puede ayudar a captar mejor las relaciones en los textos, al coste de mayor computo y memoria.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2), 
    sublinear_tf=True, 
    max_features=50_000,
    norm='l2',
    max_df=0.85,
    min_df=2,
)

A la hora de vectorizar, se ha optado por combinar el título y el texto del corpus para intentar capturar la mayor cantidad de información posible ya que el título en muchas ocasiones puede servir de resumen.

In [ ]:
corpus_combined = subcorpus['title_tfidf'].fillna('') + ' ' + subcorpus['text_tfidf']
corpus_tfidf = vectorizer.fit_transform(corpus_combined).tocsc()

Por otro lado, se combinan las consultas de train y test para obtener todas las consultas y vectorizarlas.

In [ ]:
queries = pandas.concat([train['query_tfidf'], test['query_tfidf']])
queries_tfidf = vectorizer.transform(queries).tocsc()

### Cálculo de similitudes y ranking

A continuación, definimos una función para calcular la similitud del coseno por grupos (batches) de queries para no provocar RAM allocation errors al trabajar con matrices muy grandes. Dicho esto, utilizamos argpartition para encontrar los top-k elementos de manera eficiente en dicho lote, ordenados por score.

In [43]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def rank_batch(query_matrix, corpus_tfidf, top_k=100):
    """ Gets the top k most similar documents for each query """
    # obtain similarity (query - corpus)
    scores = cosine_similarity(
        query_matrix, 
        corpus_tfidf,
        dense_output=False)
    results = []

    for i in range(scores.shape[0]):
        # for each query, get the top k results
        # get the top k indices, sorted by score
        row = scores[i]
        k = min(top_k, row.nnz)
        top_idx = np.argpartition(row.data, -k)[-k:]
        doc_idx = row.indices[top_idx]
        doc_scores = row.data[top_idx]
        order = np.argsort(-doc_scores)
        results.append(doc_idx[order])
    return results

Una vez definida esta función, en caso de que no exista un archivo con los rankings ya calculados, estos se calculan y se guardan en disco.

In [44]:
if not os.path.isfile('rankings_tfidf.npy'):
    batch_size = 256
    rankings_tfidf = []
    top_k = 100

    # rank the queries using tf-idf & save the results to disk
    for i in range(0, queries_tfidf.shape[0], batch_size):
        batch = queries_tfidf[i:i+batch_size]
        batch_rankings = rank_batch(batch, corpus_tfidf, top_k=top_k)
        rankings_tfidf.extend(batch_rankings)

    # save the ranking to a file
    normalized_ranking = []
    for ranking in rankings_tfidf:
        if ranking.shape[0] < top_k:
            # add padding to the ranking if len < 100
            ranking = np.pad(ranking, (0, top_k - ranking.shape[0]),
                constant_values=-1)
        normalized_ranking.append(ranking)
    np.save('rankings_tfidf.npy', np.array(normalized_ranking))
else:
    # load the tf-idf rankings from disk
    rankings_tfidf = np.load('rankings_tfidf.npy')

### Recuperación con embeddings

Ahora utilizamos un modelo semántico (sentence-transformers) para convertir textos en vectores densos. Este método suele capturar mejor el significado semántico, puesto que a diferencia de modelos como TF-IDF que se centran en la coincidencia exacta de palabras, estos pueden llegar a capturar similitudes entre palabras *diferentes* que tengan el mismo significado.

#### Preparación

Antes de nada, combinamos el título y texto del corpus (igual que se hizo en la sección anterior), pero en este caso con el preprocesado para embeddings.

In [34]:
corpus_combined_emb = subcorpus['title_raw'].fillna('') + ' ' + subcorpus['text_raw']

Ahora hemos de cargar el modelo que se encargará de calcular los embeddings. En concreto se hará uso de [sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2). This model maps sentences and paragraphs to a 384 dimensional dense vector space.

Es cierto que que podría haber usado un modelo específico para el idioma espanol, como lo es [hiiamsid/sentence_similarity_spanish_es](https://huggingface.co/hiiamsid/sentence_similarity_spanish_es) o uno multilingue con mas dimensiones como [sentence-transformers/paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2). Sin embargo, la capacidad de computo de mi maquina es estándar y modelos de este tipo podrían tardar bastantes horas en procesar, además de que esto es un proyecto experimental y no uno de producción.

In [35]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1271.54it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


#### Codificación del corpus

Una vez cargado el modelo, generamos los embeddings de todos los documentos del corpus. Si el archivo con el corpus codificado existe, se cargará en vez de hacer de nuevo este proceso, que puede ser lento. 

In [ ]:
if not os.path.isfile('corpus_embeddings.npy'):
    # encode the corpus using the sentence transformer model, then save to disk
    corpus_embeddings = model.encode(corpus_combined_emb.tolist(), batch_size=64, 
        show_progress_bar=True, normalize_embeddings=True)
    np.save('corpus_embeddings.npy', corpus_embeddings)
else:
    # load the corpus embeddings from disk
    corpus_embeddings = np.load('corpus_embeddings.npy')

Batches: 100%|██████████| 2943/2943 [1:02:35<00:00,  1.28s/it]


#### Codificación de consultas

Ahora, generamos los embeddings para todas las consultas (train y test). 

In [37]:
all_queries = pandas.concat([train['query_raw'], test['query_raw']]).tolist()

In [ ]:
if not os.path.isfile('query_embeddings.npy'):
    # encode the queries using the sentence transformer model
    query_embeddings = model.encode(all_queries, batch_size=64, 
        show_progress_bar=True, normalize_embeddings=True)
    np.save('query_embeddings.npy', query_embeddings)
else:
    # load the query embeddings from disk
    query_embeddings = np.load('query_embeddings.npy')

Batches: 100%|██████████| 14694/14694 [32:43<00:00,  7.48it/s]


#### Ranking con embeddings

Como los vectores están normalizados, la similitud del coseno es equivalente al producto punto (dot product). Esto nos permite calcular similitudes entre queries y corpus muy rápido multiplicando matrices, de nuevo usando batches en vez de todos los datos a la vez.

In [39]:
def rank_batch_embeddings(query_emb, corpus_emb, top_k=100):
    """ Gets the top k most similar documents for each query using embeddings """
    scores = query_emb @ corpus_emb.T
    rankings = []

    for i in range(scores.shape[0]):
        # for each query, get the top k results
        # get the top k indices, sorted by score
        row = scores[i]
        part_idx = np.argpartition(row, -top_k)[-top_k:]
        top_scores = row[part_idx]
        sorted_order = np.argsort(-top_scores)
        top_k_idx = part_idx[sorted_order]
        rankings.append(top_k_idx)
    return rankings

In [40]:
if not os.path.isfile('embedding_rankings.npy'):
    batch_size = 256
    embedding_rankings = []

    # rank the queries using embeddings & save the results to disk
    for i in range(0, query_embeddings.shape[0], batch_size):
        batch = query_embeddings[i:i+batch_size]
        batch_rankings = rank_batch_embeddings(batch, corpus_embeddings)
        embedding_rankings.extend(batch_rankings)
    np.save('embedding_rankings.npy', np.array(embedding_rankings))
else:
    # load the embedding rankings from disk
    embedding_rankings = np.load('embedding_rankings.npy')

#### Mapeo de índices a IDs

Los rankings actuales contienen índices numéricos (posiciones en el array subcorpus). Necesitamos convertirlos a los ``docid`` reales para poder evaluarlos correctamente. 

In [41]:
idx_to_docid = subcorpus['docid'].values

Convertimos rankings de TF-IDF y embeddings:

In [45]:
rankings_tdidf_final = []
for query_ranking in rankings_tfidf:
    top_docids = [idx_to_docid[idx] for idx in query_ranking]
    rankings_tdidf_final.append(top_docids)

In [46]:
rankings_emb_final = []
for query_indices in embedding_rankings:
    top_docids = [idx_to_docid[idx] for idx in query_indices]
    rankings_emb_final.append(top_docids)

### Evaluación

Por último, evaluamos ambos sistemas (TF-IDF y Embeddings) utilizando tres métricas estándar de recuperación de información: Precision@k, Recall@k y nDCG@k. Para ello, se define la siguiente función de evaluación que compara el documento correcto (y_true) con la lista de documentos recuperados (rankings).

In [47]:
import math

def evaluate(y_true, rankings, k_values=[1,5,10]):
    """ Evaluates a IR system """
    metrics_results = {k: {'precision': 0.0, 'recall': 0.0, 'ndcg': 0.0} for k in k_values}
    n_queries = len(y_true)

    for true_doc_id, predicted_docs in zip(y_true, rankings):
        # find the idx of the correct document in the list
        rank_dict = { doc: idx + 1 for idx, doc 
            in enumerate(predicted_docs) }
        rank = rank_dict.get(true_doc_id, float('inf'))

        for k in k_values:
            if rank <= k:
                k_metric = metrics_results[k]
                k_metric['precision'] += 1.0 / k
                k_metric['recall'] += 1.0
                k_metric['ndcg'] += 1.0 / math.log2(rank + 1)

    final_metrics = {}
    for k in k_values:
        k_metric = metrics_results[k]
        final_metrics[f'Precision@{k}'] = k_metric['precision'] / n_queries
        final_metrics[f'Recall@{k}'] = k_metric['recall'] / n_queries
        final_metrics[f'nDCG@{k}'] = k_metric['ndcg'] / n_queries
    return pandas.DataFrame([final_metrics])

Una vez definida la función, preparamos los datos de prueba (ground truth).

In [48]:
y_true = test['docid'].tolist()
n_train = len(train)

En primer lugar evaluamos TF-IDF, usando solo la parte de los rankings que corresponde al set de test.

In [49]:
rankings_tfidf_test = rankings_tdidf_final[n_train:]
evaluate(y_true, rankings_tfidf_test)

,Precision@1,Recall@1,nDCG@1,Precision@5,Recall@5,nDCG@5,Precision@10,Recall@10,nDCG@10
0,0.100168,0.100168,0.100168,0.051695,0.258476,0.181666,0.034316,0.343157,0.209103


De la misma manera, se evaluan los embeddings.

In [50]:
rankings_emb_test = rankings_emb_final[n_train:]
evaluate(y_true, rankings_emb_test)

,Precision@1,Recall@1,nDCG@1,Precision@5,Recall@5,nDCG@5,Precision@10,Recall@10,nDCG@10
0,0.174945,0.174945,0.174945,0.069731,0.348654,0.266843,0.041863,0.418628,0.289532


### Conclusiones 

En este trabajo se ha implementado y evaluado un sistema de recuperación de información en español utilizando el dataset Messirve y un subconjunto de Wikipedia. Se han comparado dos enfoques: un modelo léxico basado en TF-IDF y un modelo semántico basado en embeddings.

Los resultados obtenidos muestran una superioridad clara del modelo basado en embeddings frente al enfoque TF-IDF en todas las métricas evaluadas. En particular, la Precision@1 pasa de 0.100 en TF-IDF a 0.175 en embeddings, lo que supone un incremento notable en la proporción de consultas cuyo documento correcto aparece en primera posición. Asimismo, el Recall@10 aumenta de 0.343 a 0.419, lo que indica que el modelo semántico recupera el documento relevante dentro del top-10 en un mayor número de casos. De forma coherente, el nDCG@10 también mejora de 0.209 a 0.290, reflejando una mejor ordenación de los resultados.

Estos resultados confirman que el modelo de embeddings captura mejor la similitud semántica entre consultas y documentos que el modelo léxico basado en coincidencia de términos. Mientras que TF-IDF depende de coincidencias exactas de palabras, los embeddings permiten generalizar a sinónimos y variaciones, lo que da mucha mayor flexibilidad.

Como posibles mejoras, se propone explorar enfoques híbridos que combinen modelos sparse y dense, utilizar modelos de embeddings más avanzados, aplicar técnicas de re-ranking con cross-encoders o usar modelos más complejos y específicos para el idioma espanol como se comentó en este notebook.